<a href="https://colab.research.google.com/github/VipanchiKatthula/TwitterSentimentAnalysis/blob/master/sent_twit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from string import punctuation
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
sno = nltk.stem.SnowballStemmer('english')
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import coo_matrix, hstack
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [4]:
!pip install -q xlrd
!git clone https://github.com/VipanchiKatthula/TwitterSentimentAnalysis.git

Cloning into 'TwitterSentimentAnalysis'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 10 (delta 1), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (10/10), done.


In [0]:
linelist1=[]
linelist2=[]
target1 = []
target2 = []
with open ('TwitterSentimentAnalysis/train_90.txt',encoding="utf8") as t1:
    for line in t1:
        linelist1.append(word_tokenize((line.split(',',4)[3].lstrip())))
        target1.append(line.split(',', 4)[1])
with open ('TwitterSentimentAnalysis/test_10.txt',encoding="utf8") as t2:
    for line in t2:
        linelist2.append(word_tokenize((line.split(',',4)[3].lstrip())))
        target2.append(line.split(',', 4)[1])        

In [0]:
print(linelist1[1:4])
print(linelist2[1:4])

[['is', 'so', 'sad', 'for', 'my', 'APL', 'friend', '...', '...', '...', '...', '.'], ['I', 'missed', 'the', 'New', 'Moon', 'trailer', '...'], ['omg', 'its', 'already', '7:30', ':', 'O']]
[['``', '@', 'CleverMonkeys', 'but', 'AFTER', 'you', 'hit', 'up', 'the', 'Magnolia', 'to', 'see', 'ANVIL', '!', 'THE', 'STORY', 'OF', 'ANVIL', 'and', 'the', 'band', 'playing', 'a', 'small', 'set', 'shortly', 'after'], ['@', 'CleverMonkeys', 'only', 'seen', 'a', 'boot', 'on', 'a', 'car', 'in', 'movies', '!', 'goodthing', 'I', "'m", 'a', 'vegetarian', 'I', 'will', 'only', 'be', 'eatting', 'beans', 'until', 'the', 'end', 'of', 'month', '!'], ['@', 'clevertitania', 'Good', 'morning', '.', 'We', 'have', 'rain', 'and', 'thunder', 'here']]


In [0]:
all_tweets_train = []
list_stop_words = set(stopwords.words('english'))
for i in range (1,len(linelist1)):
    token = linelist1[i]
    filtoken=[]
    for word in token:
        if word not in list_stop_words:
            if word.isalpha():
                filtoken.append(sno.stem(word.lower()))
    all_tweets_train.append(filtoken)

all_tweets_test = []
for i in range (1,len(linelist2)):
    token = linelist2[i]
    filtoken=[]
    for word in token:
        if word not in list_stop_words:
            if word.isalpha():
                filtoken.append(sno.stem(word.lower()))
    all_tweets_test.append(filtoken)
    


In [0]:
a = all_tweets_train
b = all_tweets_test
lis1 = []
lis2 = []
def listToString(s):  
    str1 = " "    
    return (str1.join(s))              
for ele in a:
    strg = listToString(ele)
    lis1.append(strg)
for ele in b:
    strg = listToString(ele)
    lis2.append(strg)


In [0]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=10000)
train_vectors = pd.DataFrame( data = vectorizer.fit_transform(lis1).todense(),columns = vectorizer.get_feature_names())
test_vectors = pd.DataFrame( data = vectorizer.transform(lis2).todense(),columns = vectorizer.get_feature_names())


In [0]:
target1.pop(0)
target2.pop(0)
target1 = [float(i) for i in target1]
target2 = [float(i) for i in target2]
y_label = pd.DataFrame( data = target1)
y_test = pd.DataFrame( data = target2)


In [0]:
class LogisticRegression:
    def __init__(self, alp=0.01, niter=1000):
        self.lr = alp
        self.niter = niter
    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))   #test.values.astype(float)

    def fit(self, X, y):
        no_of_cols = 10000#X.shape[1]
        self.theta = np.zeros((no_of_cols,1))
        for i in range(self.niter):
            z = np.dot(X, self.theta)
            y_out = self.sigmoid(z)
            y_out = y_out.reshape(y.shape)
            gradient = np.dot(X.T, (y_out - y)) / y.size
            self.theta -= self.lr * gradient
            
            if(i % 10000 == 0):
                z = np.dot(X, self.theta)
                y_out = self.sigmoid(z)

    def predict_prob(self, X):
        return self.sigmoid(np.dot(X, self.theta))
    def predict(self, X, th):
        label_array = np.array([])
        pred = self.predict_prob(X)
        
        for i in pred:
          if i > th:
            label_array = np.append(label_array,1)
          else:
            label_array = np.append(label_array,0)
        return label_array

In [0]:
X = train_vectors.to_numpy()
X_test = test_vectors.to_numpy()

In [0]:
model = LogisticRegression()
model.fit(X, y)

In [18]:
kf = KFold(n_splits=10)
sc = np.array([])
count = 1
for train_index, test_index in kf.split(train_vectors):
  x_train, x_test = train_vectors.iloc[train_index],train_vectors.iloc[test_index]
  y_train, y_test = y_label.iloc[train_index],y_label.iloc[test_index]
  logreg_model = LogisticRegression()
  print("Runing model number:",count)
  logreg_model.fit(x_train.to_numpy(),y_train.to_numpy())
  pred = logreg_model.predict_prob(x_test.to_numpy())
  th = sum(pred)/len(pred)
  labels = logreg_model.predict(x_test,th)
  print("completed!! model number:",count)
  sc = np.append(sc,accuracy_score(y_test,labels))
  count +=1
print(sc)


 

Runing model number: 1
completed!! model number: 1
Runing model number: 2
completed!! model number: 2
Runing model number: 3
completed!! model number: 3
[0.64986667 0.6228     0.61456667]


<generator object _BaseKFold.split at 0x7fa336799bf8>

In [0]:
test_out = model.predict_prob(X_test)

In [0]:
sum(test_out)/len(test_out>0.53)

array([0.50130522])

In [0]:
count = 0
rcount =0
for i in test_out:
  if i< 0.50130522:
    count += 1
  else: rcount +=1



In [0]:
print(count,rcount)

6552 3448
